In [1]:
import pandas as pd
import janitor

In [2]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [3]:
data = pd.read_csv('filter.csv', encoding='utf-8')

In [4]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [5]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [6]:
len(data)

181772

### Note
Daily report for april 20 (ie, reported in 21 april) stops using testing site (ENTIDAD_UM),
<br>
And starts using region of residence (ENTIDAD_RES) so we shall treat both sets of data accordingly

In [7]:
dat = data.copy()

In [8]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [9]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-21','2022-12-31') # the long con
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [10]:
tbl = pd.concat([tbl_x, tbl_y])

In [11]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,10
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,9
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,2
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,7
...,...,...,...,...,...,...,...
4907,2020-04-28,2020-04-27,9999-99-99,29,089695,Local,1
4908,2020-04-28,2020-04-28,9999-99-99,14,0a8226,Local,1
4909,2020-04-29,2020-04-15,9999-99-99,2,02f6ab,Local,1
4910,2020-04-29,2020-04-21,9999-99-99,9,0f9fb6,Local,1


In [12]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [13]:
tbl['Status'] = '1'

In [43]:
tbl

,Date_Confirmed,Date_Symptoms,FECHA_DEF,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,1
...,...,...,...,...,...,...,...
4907,2020-04-28,2020-04-27,9999-99-99,29,089695,Local,1
4908,2020-04-28,2020-04-28,9999-99-99,14,0a8226,Local,1
4909,2020-04-29,2020-04-15,9999-99-99,2,02f6ab,Local,1
4910,2020-04-29,2020-04-21,9999-99-99,9,0f9fb6,Local,1


In [44]:
#tbl[~tbl.FECHA_DEF.str.startswith(('9999'))]
tbl = tbl[tbl.FECHA_DEF.str.startswith(('9999'))] # Not deceased

In [45]:
tbl = tbl.drop(['FECHA_DEF'], axis=1)

In [46]:
tbl

,Date_Confirmed,Date_Symptoms,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,15,1879e0,Local,1
4,2020-02-07,2020-02-07,11,03e780,Local,1
...,...,...,...,...,...,...
4907,2020-04-28,2020-04-27,29,089695,Local,1
4908,2020-04-28,2020-04-28,14,0a8226,Local,1
4909,2020-04-29,2020-04-15,2,02f6ab,Local,1
4910,2020-04-29,2020-04-21,9,0f9fb6,Local,1


In [47]:
#tbl.to_csv('confirmed_by_date.csv', index=False)

In [48]:
geo = pd.read_csv('../../data/geo/entidades.csv')

In [49]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [50]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [51]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [52]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Region', 'Region_ID', 'Region_Key', 'Origin']]

In [53]:
output

,Date_Confirmed,Date_Symptoms,Region,Region_ID,Region_Key,Origin
0,2020-01-06,2020-01-06,NAYARIT,18,NT,Local
1,2020-03-18,2020-03-17,NAYARIT,18,NT,Local
2,2020-03-19,2020-03-10,NAYARIT,18,NT,Local
3,2020-03-23,2020-03-19,NAYARIT,18,NT,Local
4,2020-03-24,2020-03-21,NAYARIT,18,NT,Local
...,...,...,...,...,...,...
17324,2020-04-28,2020-04-24,TLAXCALA,29,TL,Local
17325,2020-04-28,2020-04-25,TLAXCALA,29,TL,Local
17326,2020-04-28,2020-04-26,TLAXCALA,29,TL,Local
17327,2020-04-28,2020-04-26,TLAXCALA,29,TL,Local


In [54]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACÁN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [55]:
output.Region.unique()

array(['NAYARIT', 'VERACRUZ', 'AGUASCALIENTES', 'MÉXICO', 'GUANAJUATO',
       'BAJA CALIFORNIA', 'CIUDAD DE MÉXICO', 'COAHUILA', 'SINALOA',
       'CHIAPAS', 'YUCATÁN', 'MORELOS', 'NUEVO LEÓN', 'QUERÉTARO',
       'PUEBLA', 'DURANGO', 'JALISCO', 'QUINTANA ROO', 'CAMPECHE',
       'SAN LUIS POTOSÍ', 'GUERRERO', 'OAXACA', 'COLIMA', 'CHIHUAHUA',
       'SONORA', 'TAMAULIPAS', 'HIDALGO', 'TABASCO', 'MICHOACAN',
       'BAJA CALIFORNIA SUR', 'ZACATECAS', 'TLAXCALA'], dtype=object)

In [56]:
output.Origin.unique()

array(['Local', 'Estados Unidos de AmÃ©rica', 'Estados Unidos de América',
       'Venezuela', 'PerÃº', 'Perú', 'El Salvador', 'Brasil', 'Cuba',
       'Guatemala', 'Canadá', 'Alemania', 'RepÃºblica de Honduras',
       'República de Honduras', 'Otro', 'CamerÃºn', 'Camerún', 'Chile'],
      dtype=object)

In [57]:
output.to_csv('../../latest.csv', index=False)